In [29]:
import boto
from boto.manage.cmdshell import sshclient_from_instance
import time
import os

## S3 Management

Main Source : https://aws.amazon.com/articles/3998

In [4]:
s3 = boto.connect_s3(aws_access_key_id='xxx', 
                     aws_secret_access_key='xxx')

# An example : creation of a bucket + import in S3 a local file
bucket = s3.create_bucket('harvard_practicals1')
key = bucket.new_key('data/NB.rtf')
key.set_contents_from_filename('NB.rtf')
key.set_acl('public-read')

## EC2 Connection

In [9]:
ec2 = boto.connect_ec2(aws_access_key_id='xxx', 
                       aws_secret_access_key='xxx')


True

##### To Do Only Once : Key-Pair Setup + Group Creation

In [ ]:
# ##To do Only Once : 
# key_pair = ec2.create_key_pair('ec2-romain2-key')
# key_pair.save('~/.ssh')
# group = ec2.create_security_group('tag', 'A group that allows SSH access')
# group.authorize('tcp', 22, 22, '0.0.0.0/0') # to properly authorize ssh

In [77]:
## Running a classical Amazon t2.micro Linux Instance
reservation = ec2.run_instances(image_id='ami-60b6c60a', 
                                key_name='ec2-romain2-key', 
                                instance_type = 't2.micro',
                                security_groups = ['tag'])

instance = reservation.instances[0]
print 'waiting for instance'
while instance.state != 'running':
    print '.',
    time.sleep(5)
    instance.update()
print 'done'

waiting for instance
. . . . . done


## Upload a file on EC2

In [41]:
import boto.manage.cmdshell
 
def upload_file(instance, key, username, local_filepath, remote_filepath):
    """
    Upload a file to a remote directory using SFTP. All parameters except
    for "instance" are strings. The instance parameter should be a
    boto.ec2.instance.Instance object.
 
    instance        An EC2 instance to upload the files to.
    key             The file path for a valid SSH key which can be used to
                    log in to the EC2 machine.
    username        The username to log in as.
    local_filepath  The path to the file to upload.
    remote_filepath The path where the file should be uploaded to.
    """
    ssh_client = boto.manage.cmdshell.sshclient_from_instance(
        instance,
        key,
        user_name=username
    )
    ssh_client.put_file(local_filepath, remote_filepath)

In [63]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain2-key.pem', 
            'ec2-user', 
            '/Users/romainbui/test', 'test')

## Run Command lines on EC2

In [79]:
ssh_client = boto.manage.cmdshell.sshclient_from_instance(instance, 
                                                          '/Users/romainbui/.ssh/ec2-romain2-key.pem', 
                                                          user_name = 'ec2-user')

##### Perform the instalation

In [80]:
main_install =  """sudo yum -y install gcc-c++ python27-devel atlas-sse3-devel lapack-devel
sudo pip install numpy
sudo pip install pandas
sudo pip install scipy
sudo pip install scikit-learn
"""

ssh_client.run_pty(main_install)

<paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0x35f8650L (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>

## Instance Termination | DO NOT FORGET !

In [81]:
# Terminate the Instance - DO NOT FORGET
instance.terminate()